In [13]:
import pandas as pd

# Load dataset
df = pd.read_csv('/content/generated_inventory_dataset.csv')

# Display the first few rows of the dataset
print(df.head())


  RegionName                  CountryName          State           City  \
0   Missouri                      Bahamas    Mississippi  Martinezshire   
1  Wisconsin                        Kenya   South Dakota   Barajasburgh   
2   Nebraska  French Southern Territories       Delaware    Johnchester   
3    Arizona                     Cambodia        Florida    Anthonytown   
4   Kentucky                        Samoa  Massachusetts   Charlesmouth   

   PostalCode                                   WarehouseAddress  \
0       50167  6818 Robinson Greens Apt. 594\nJesusport, MA 6...   
1        8169  6711 Gonzalez Run Apt. 345\nLucaschester, IL 7...   
2       82563  39749 Marisa Brooks Apt. 138\nSantosmouth, NY ...   
3       47333  770 Daniels Plaza Suite 597\nLake Amber, SC 06983   
4       28711           93041 Rogers Mall\nDouglasfurt, NH 17390   

                  WarehouseName      EmployeeName               EmployeeEmail  \
0                     Davis Inc      Steven Perez    youngj

In [30]:
import heapq

class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = {}
        self.distances = {}

    def add_node(self, value):
        self.nodes.add(value)
        self.edges[value] = []

    def add_edge(self, from_node, to_node, distance):
        self.edges[from_node].append(to_node)
        self.distances[(from_node, to_node)] = distance

def dijkstra(graph, initial):
    visited = {initial: 0}
    path = {}

    nodes = set(graph.nodes)

    while nodes:
        min_node = None
        for node in nodes:
            if node in visited:
                if min_node is None:
                    min_node = node
                elif visited[node] < visited[min_node]:
                    min_node = node

        if min_node is None:
            break

        nodes.remove(min_node)
        current_weight = visited[min_node]

        for edge in graph.edges[min_node]:
            weight = current_weight + graph.distances[(min_node, edge)]
            if edge not in visited or weight < visited[edge]:
                visited[edge] = weight
                path[edge] = min_node

    return visited, path

# Create graph from dataset
graph = Graph()
for node in pd.concat([df['Origin_Location'], df['Destination_Location']]).unique():
    graph.add_node(node)

# Assuming df has columns 'Origin_Location', 'Destination_Location', and 'TRANSPORTATION_DISTANCE_IN_KM'
for _, row in df.iterrows():
    graph.add_edge(row['Origin_Location'], row['Destination_Location'], row['TRANSPORTATION_DISTANCE_IN_KM'])

start_location = 'North Lisaton'  # Replace with actual starting location
distances, paths = dijkstra(graph, start_location)
print(distances)


{'North Lisaton': 0, 'Derrickton': 1586.94}


In [31]:
import random
import pandas as pd

# Load dataset
df = pd.read_csv('/content/generated_inventory_dataset.csv')

class GeneticAlgorithm:
    def __init__(self, population_size, mutation_rate, generations):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.generations = generations

    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            individual = self.create_individual()
            population.append(individual)
        return population

    def create_individual(self):
        # Create a random individual (solution)
        # Example: A list of route indices
        individual = df.sample(frac=1).index.tolist()
        return individual

    def fitness(self, individual):
        # Fitness function to evaluate the individual
        # Example: Total distance of the route
        total_distance = sum(df.loc[individual, 'TRANSPORTATION_DISTANCE_IN_KM'])
        return 1 / total_distance

    def selection(self, population):
        # Select individuals based on fitness (roulette wheel selection)
        fitnesses = [self.fitness(ind) for ind in population]
        total_fitness = sum(fitnesses)
        probs = [fit / total_fitness for fit in fitnesses]
        selected = random.choices(population, weights=probs, k=2)
        return selected

    def crossover(self, parent1, parent2):
        # Single point crossover
        point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2

    def mutate(self, individual):
        # Randomly swap two elements in the individual
        if random.random() < self.mutation_rate:
            idx1, idx2 = random.sample(range(len(individual)), 2)
            individual[idx1], individual[idx2] = individual[idx2], individual[idx1]

    def run(self):
        population = self.initialize_population()
        for generation in range(self.generations):
            new_population = []
            for _ in range(self.population_size // 2):
                parent1, parent2 = self.selection(population)
                child1, child2 = self.crossover(parent1, parent2)
                self.mutate(child1)
                self.mutate(child2)
                new_population.extend([child1, child2])
            population = new_population
            best_individual = max(population, key=self.fitness)
            print(f'Generation {generation}: Best fitness = {self.fitness(best_individual)}')
        return best_individual

# Example usage
ga = GeneticAlgorithm(population_size=50, mutation_rate=0.01, generations=10)
best_route = ga.run()
print(f'Optimized Route: {best_route}')


Generation 0: Best fitness = 9.948588777261633e-07
Generation 1: Best fitness = 9.98712918713137e-07
Generation 2: Best fitness = 1.0007882408342471e-06
Generation 3: Best fitness = 1.0122293911006983e-06
Generation 4: Best fitness = 9.984233497514736e-07
Generation 5: Best fitness = 9.976779643985408e-07
Generation 6: Best fitness = 1.0136378690728975e-06
Generation 7: Best fitness = 1.02602243237989e-06
Generation 8: Best fitness = 1.0033794522316785e-06
Generation 9: Best fitness = 1.0080116971290155e-06
Optimized Route: [240, 317, 751, 866, 453, 325, 808, 196, 519, 12, 171, 550, 804, 95, 298, 592, 52, 518, 241, 112, 249, 598, 998, 15, 734, 150, 923, 589, 51, 357, 43, 981, 835, 899, 969, 908, 223, 163, 392, 405, 175, 695, 796, 892, 495, 452, 265, 768, 78, 158, 635, 659, 109, 802, 658, 962, 174, 525, 377, 415, 836, 204, 932, 586, 188, 73, 564, 212, 521, 720, 341, 745, 468, 29, 20, 473, 561, 338, 557, 988, 474, 538, 460, 813, 597, 334, 648, 526, 611, 425, 123, 556, 523, 877, 773, 40, 

In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import pandas as pd

# Load dataset
df = pd.read_csv('/content/generated_inventory_dataset.csv')

# One-hot encode categorical features
X = pd.get_dummies(df[['Origin_Location', 'Destination_Location']])
y = df['TRANSPORTATION_DISTANCE_IN_KM']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')


Mean Absolute Error: 483.41889900000007
